# 원본 데이터 저장

In [1]:
import requests
import json
from collections import Counter

req = requests.get('http://52.231.65.246:9200/test/news/_search/?size=10000')

def read_json(req):
    json_dict = json.loads(req.content.decode())

    body_ls = []
    category_ls = []
    try:
        for article in json_dict['hits']['hits']:
            body = article['_source']['body'].replace('\t',' ').replace('\n',' ')
            body_ls.append(body)

            category = article['_source']['category']
            category_ls.append(category)
    except:
        pass
    
    return body_ls, category_ls

body_ls, category_ls = read_json(req)

In [2]:
Counter(category_ls)

Counter({'IT': 156,
         '경제': 1539,
         '기타': 403,
         '사회': 3399,
         '생활': 819,
         '세계': 738,
         '스포츠': 633,
         '연예': 433,
         '오피니언': 619,
         '정치': 1261})

In [244]:
with open('news.txt', 'w') as f:
    for category, body in zip(category_ls, body_ls):
        f.write('%s\t%s\n'%(category, body))

# 전처리 + 토크나이징

In [4]:
def read_txt(path_to_file):
    body_ls = []
    category_ls = []
    
    with open(path_to_file, 'r') as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            try:
                category, body = line.split('\t')
                category_ls.append(category)
                body_ls.append(body)
            except:
                print(i)
    return body_ls, category_ls

In [5]:
body_ls, category_ls = read_txt('news.txt')

In [25]:
# 전처리(클렌징)
import re

# 형태소 분석
from konlpy.tag import Okt
import konlpy

# 병렬처리
from multiprocessing import Process,Queue, Pool
import functools
from threading import Thread
from konlpy import jvm
import jpype
import queue

In [42]:
class Preprocessor(object):

    def __init__(self):
        '''
        전처리를 위한 함수들이 저장된 클래스입니다.
        '''

        self.twit = Okt()# 한 개의 문서에서 명사(noun)만 추출하는 함수
        
        # 정규 표현식 리스트
        self.regex_ls = [
                    '[\t\n\r\f\v]', #공백 제거,
            '\(.+?\)', '\[.+?\]', '\<.+?\>',  '◀.+?▶',  '=.+=', #특수문자 사이에 오는 단어 제거
            '(?<=▶).+', '(?<=▷).+', '(?<=※).+', #특수문자 다음으로 오는 단어 제거
            #'(?<=\xa0).+', # \xa0(증권 기사) 다음으로 오는 단어 제거
            '(?<=\Copyrights).+', # Copyrights 다음에 오는 기사 제거
            '[\w]+@[a-zA-Z]+\.[a-zA-Z]+[\.]?[a-z]*', #이메일 제거
            '[가-힣]+ 기자', '[가-힣]+ 선임기자',
            '[\{\}\[\]\/?,;·:“‘|\)*~`!^\-_+<>@○▲▶■◆\#$┌─┐&\\\=\(\'\"├┼┤│┬└┴┘|ⓒ]', #특수문자 제거
            #'[0-9]+[년월분일시조억만천원]*' , #숫자 단위 제거
        ]
        
        # 2. 제거 대상 단어 리스트
        self.word_to_be_cleaned_ls = ['한기재','헬스경향','디지털뉴스팀', '\u3000','Copyrights','xa0', 
                                      'googletagdisplay', 'windowjQuery', 'documentwrite']
        
        # 3. 불용어 리스트
        self.stopword_ls = ['에서','으로','했다','하는','이다','있다','하고','있는','까지','이라고','에는',
                            '한다','지난','관련','대한','됐다','부터','된다','위해','이번','통해','대해',
                            '애게','했다고','보다','되는','에서는','있다고','한다고','하기','에도','때문',
                            '하며','하지','해야','이어','하면','따라','하면서','라며','라고','되고','단지',
                            '이라는','이나','한다는','있따는','했고','이를','있도록','있어','하게','있다며',
                            '하기로','에서도','오는','라는','이런','하겠다고','만큼','이는','덧붙였다','있을',
                            '이고','이었다','이라','있으며','있고','이며','했다며','됐다고','나타났다','한다며',
                            '하도록','있지만','된다고','되면서','그러면서','그동안','해서는','에게','밝혔다', '한편',
                            '최근', '있다는','보이','되지','정도','지난해','매년','오늘','되며','하기도', '지난달',
                            '하겠다는','했다세라','올해', '바로', '바랍니다', '함께','이후','따르면','같은','오후','모두',
                            '로부터','전날','면서','했다는','그리고','있던'
                           ]

    # 크롤링한 text에 정규표현식을 적용하는 함수입니다.
    def clean_text(self,text):
        try:
            for regex in self.regex_ls:
                text = re.sub(regex, '', text)
        except:
            text = ' '
        return text

    # 복수 개의 문서를 클렌징하는 함수입니다.
    def clean_doc(self, doc_ls):
        '''
        정규표현식으로 문서를 전처리하는 함수입니다.
        
        input
        doc_ls : list(iterable)
            str형태의 text를 원소로 갖는 list type
        '''
        return [self.clean_text(doc) for doc in doc_ls]


    # 불용어를 제거하는 함수입니다.
    def remove_stopwords(self, token_doc_ls):
        '''
        불용어를 제거하는 함수입니다.

        input
        token_doc_ls : str, iterable
            token 형태로 구분된 문서가 담긴 list 형식
        '''

        total_stopword_set = set(self.stopword_ls)

        # input이 복수 개의 문서가 담긴 list라면, 개별 문서에 따라 단어를 구분하여 불용어 처리
        return_ls = []

        if token_doc_ls:
            if type(token_doc_ls[0]) == list:
                for doc in token_doc_ls:
                    return_ls += [[token for token in doc if not token in total_stopword_set]]

            elif type(token_doc_ls[0]) == str:
                return_ls = [token for token in token_doc_ls if not token in total_stopword_set]

        return return_ls  
    
    def _extract_nouns_for_single_doc(self, doc):
        '''
        명사 추출
        '''
        clean_doc = self.clean_text(doc) # 클렌징
        token_ls = [x for x in self.twit.nouns(clean_doc) if len(x) > 1] # 토크나이징
        return self.remove_stopwords(token_ls) # 불용어 제거


    # 한 개의 문서에서 형태소(morphs)만 추출하는 함수
    def _extract_morphs_for_single_doc(self, doc):
        '''
        형태소 추출
        '''
        clean_doc = self.clean_text(doc) # 클렌징
        token_ls = [x for x in self.twit.morphs(clean_doc) if len(x) > 1] # 토크나이징
        return self.remove_stopwords(token_ls) # 불용어 제거



    # 모든 문서에서 명사(nouns)을 추출하는 함수.
    def extract_nouns_for_all_document(self,doc_ls, stopword_ls = []):
        '''
        모든 문서에서 명사를 추출하는 함수입니다.
        전처리를 적용하고 불용어를 제거한 결과를 반환합니다.

        input
        doc_ls : iterable, 원문이 str형태로 저장된 list

        return
        전처리 적용, 불용어 제거
        list : 각각의 문서가 토크나이징 된 결과를 list형태로 반환
        '''
        jpype.attachThreadToJVM()
        # 전처리
        clean_doc_ls = self.clean_doc(doc_ls)

        # 명사 추출
        token_doc_ls = [self._extract_nouns_for_single_doc(doc) for doc in clean_doc_ls]

        # 불용어 제거
        return self.remove_stopwords(token_doc_ls)



    # 모든 문서에서 형태소(morph)를 추출하는 함수.
    def extract_morphs_for_all_document(self,doc_ls, stopword_ls = []):
        '''
        모든 문서에서 형태소(morph)를 추출하는 함수입니다.
        전처리를 적용하고 불용어를 제거한 결과를 반환합니다.

        input
        doc_ls : iterable, 원문이 str형태로 저장된 list

        return
        list : 각각의 문서가 토크나이징 된 결과를 list형태로 반환
        '''
        jpype.attachThreadToJVM()
        # 전처리
        clean_doc_ls = self.clean_doc(doc_ls)

        # 형태소(morph) 추출
        token_doc_ls = [self._extract_morphs_for_single_doc(doc) for doc in clean_doc_ls]

        # 불용어 제거
        return self.remove_stopwords(token_doc_ls)

    
    
    # 토크나이징을 병렬처리 하는데 사용되는 함수
    def _extract_nouns_for_multiprocessing(self, tuple_ls):
        jpype.attachThreadToJVM()
        # 멀티프로세싱의 경우, 병렬처리시 순서가 뒤섞이는 것을 방지하기위해,
        # [(idx, doc)] 형태의 tuple이 들어온다.
        return [(idx, self._extract_nouns_for_single_doc(doc)) for idx, doc in tuple_ls]

    # 토크나이징을 병렬처리 하는데 사용되는 함수
    def _extract_morphs_for_multiprocessing(self, tuple_ls):
        jpype.attachThreadToJVM()
        # 멀티프로세싱의 경우, 병렬처리시 순서가 뒤섞이는 것을 방지하기위해,
        # [(idx, doc)] 형태의 tuple이 들어온다.
        return [(idx, self._extract_morphs_for_single_doc(doc)) for idx, doc in tuple_ls]

    # 토크나이징을 병렬처리 하기 위해, 작업을 분할하는 함수
    def split_list(self, ls, n):
        '''
        병렬처리를 위해, 리스트를 원하는 크기(n)로 분할해주는 함수입니다.
        '''
        result_ls = []
        for i in range(0, len(ls), n):
            result_ls += [ls[i:i+n]]
        
        return result_ls

    def extract_morphs_for_all_document_FAST_VERSION(self,
                                                     doc_ls,
                                                     n_thread = 4):
        jpype.attachThreadToJVM()

        '''
        멀티쓰레딩을 적용하여 속도가 개선된 버전입니다.
        문서들을 전처리하고 불용어(stopwords)를 제거한 후, Tokenzing하는 함수입니다.

        inputs
        doc_ls : iterable, 원문이 str 형태로 담겨있는 list를 input으로 받습니다.
        n_thread: int[default : 4], 사용하실 쓰레드의 갯수를 input으로 받습니다.
        '''

        # 텍스트 클렌징 작업 수행
        # [(idx, clean_doc)] 형태로 저장 (나중에 sorting을 위해)
        clean_tuple_ls = [(idx, clean_doc) for idx, clean_doc in zip(range(len(doc_ls)), self.clean_doc(doc_ls))]

        # 멀티쓰레딩을 위한 작업(리스트)분할
        length = len(clean_tuple_ls)
        splited_clean_tuple_ls = self.split_list(clean_tuple_ls, length//n_thread)

        que = queue.Queue()
        thread_ls = []

        for tuple_ls in splited_clean_tuple_ls:

            temp_thread = Thread(target= lambda q, arg1: q.put(self._extract_morphs_for_multiprocessing(arg1)),  args = (que, tuple_ls))

            temp_thread.start()
            thread_ls.append(temp_thread)

        for thread in thread_ls:
            thread.join()

        # 정렬을 위한 index_ls와 token_ls를 사용
        index_ls = []
        token_ls = []

        # thread의 return 값을 결합
        while not que.empty():

            result = que.get() # [(idx, token), (idx, token)...] 형태를 반환
            index_ls += [idx for idx, _ in result]
            token_ls += [token for _, token in result]

        token_ls = [token for idx, token in sorted(zip(index_ls, token_ls))]  
        token_ls = [' '.join(tokens) for tokens in token_ls]
    
        return token_ls


In [43]:
pre = Preprocessor()

In [44]:
%%time
clean_body_ls = pre.extract_morphs_for_all_document_FAST_VERSION(body_ls)

CPU times: user 8min 17s, sys: 16.1 s, total: 8min 33s
Wall time: 2min 15s


In [75]:
k = 306
body_ls[k]

'파울루 벤투호에 패한 오스카 타바레스 우루과이 대표팀 감독이 한국 축구는 많이 성장했고 강해졌다”며 박수를 보냈다.벤투 감독이 이끄는 축구대표팀은 12일 오후 서울월드컵경기장에서 열린 우루과이와의 평가전에서 21로 승리했다. 후반 20분 황의조의 선제골과 후반 34분 정우영의 결승골이 승전고를 울렸다.경기 후 일흔이 넘은 노장 타바레스 감독은 좋은 경기였다. 전반전은 양팀이 서로 균형이 맞았다”며 결과를 가져오지 못해 아쉬움은 있으나 좋은 경기를 펼쳤다는 것에 만족한다”고 소감을 밝혔다.타바레스 감독은 우루과이와 한국은 12시간의 시차가 있고 또 장시간 비행으로 인해 체력적인 어려움도 있었다”고 정상적 컨디션이 아니었다는 것을 이야기하면서도 러시아 월드컵과 견줘 한국 축구는 많이 성장했고 강해졌다. 특히 손흥민은 톱 클래스의 플레이를 펼치고 있다. 한국은 더 성장할 여지가 있다”고 평가했다.벤투 감독에 대해서는 그와 한 시즌을 같이 생활했는데 당시에도 이미 높은 수준을 보여주고 있었기에 그를 잊을 수 없다”며 감독으로도 잘 성장하고 있다. 중국과 포르투갈 그리고 한국에서 커리어를 쌓고 있는데 이대로 걸어간다면 세계적인 명장이 될 수 있을 것”이라고 덕담했다.타바레스 감독은 199798시즌 스페인의 레알 오비에도를 이끌던 시절 선수 벤투와 연을 맺었다.타바레스 감독은 끝으로 우루과이는 세대교체 중이다. 오늘 경기만 가지고 과거의 우루과이와 비교하는 것은 무리다. 러시아 월드컵은 끝났고 이제 새로운 출발 앞에 서 있다”면서 지금은 카타르 월드컵에 집중하는 단계다. 잘 준비해 다음 주에 있을 일본전을 대비할 것”이라고 밝혔다.박태근  '

In [76]:
clean_body_ls[k]

'파울루 투호 오스카 타바레스 우루과이 대표팀 감독 한국 축구 많이 성장했고 강해졌다 박수 보냈다 감독 이끄는 축구 대표팀 12일 월드컵경기 열린 우루과이 와의 평가전 21 승리 후반 20분 황의조 선제골 후반 34분 정우영 결승골 승전 고를 울렸다 경기 일흔 넘은 노장 타바레스 감독 좋은 경기 였다 전반전 서로 균형 맞았다 결과 가져오지 아쉬움 있으나 좋은 경기 펼쳤다는 만족한다 소감 타바레스 감독 우루과이 한국 12시간 시차 장시간 비행 인해 체력 어려움 있었다 정상 컨디션 아니었다는 이야기 하면서도 러시아 월드컵 견줘 한국 축구 많이 성장했고 강해졌다 특히 손흥민 클래스 플레이 펼치고 한국 성장할 여지 평가 감독 서는 시즌 같이 생활 했는데 당시 이미 높은 수준 보여주고 있었기에 잊을 없다 감독 으로도 성장하고 중국 포르투갈 한국 커리어 쌓고 있는데 이대로 걸어간다면 세계 명장 덕담 타바레스 감독 199798시 스페인 레알 오비에도 이끌던 시절 선수 맺었다 타바레스 감독 우루과이 세대 교체 중이 경기 가지 과거 우루과이 비교 무리 러시아 월드컵 끝났고 이제 새로운 출발 지금 카타르 월드컵 집중 단계 준비 다음 본전 대비 박태'

In [77]:
category_ls[k]

'스포츠'

In [47]:
token_ls = [token for tokens in clean_body_ls for token in tokens.split()]

In [78]:
Counter(token_ls).most_common(10)

[('한국', 7604),
 ('미국', 6335),
 ('중국', 6064),
 ('서울', 5360),
 ('대통령', 5052),
 ('정부', 5049),
 ('사업', 4155),
 ('문제', 4124),
 ('지역', 4061),
 ('사람', 3958)]

In [79]:
with open('news_tokenized.txt', 'w') as f:
    for category, body in zip(category_ls, clean_body_ls):
        f.write('%s\t%s\n'%(category, body))

# 단어 임베딩

In [239]:
# Y 더미화
from collections import defaultdict

# CBOW 계산
import numpy as np

# 임베딩
import os
from gensim.models import Word2Vec, Doc2Vec

# 분류기 학습 결과 계산
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

# 결과 저장
import pandas as pd

In [240]:
def read_txt(path_to_file):
    body_ls = []
    category_ls = []
    
    with open(path_to_file, 'r') as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            try:
                category, body = line.split('\t')
                category_ls.append(category)
                
                body_ls.append(body)
            except:
                print(i)
    return body_ls, category_ls

In [241]:
def make_cbow(token_ls, word2vec):
    cbow_ls = []

    for tokens in token_ls:
        cbow = np.zeros((word2vec.vector_size))

        for token in tokens:
            cbow += word2vec.wv.get_vector(token)

        cbow_ls.append(cbow)
    return cbow_ls

## **Word2Vec train**

In [ ]:
def main():
    # data load
    token_ls, category_ls = read_txt('news_tokenized.txt')
    token_ls = [tokens.split() for tokens in token_ls]

    # train test split
    train_size = int(round(len(token_ls) * 0.8))

    x_train = token_ls[:train_size]
    y_train = category_ls[:train_size]

    x_test = token_ls[train_size:]
    y_test = category_ls[train_size:]
    
    # Y 더미화
    cat2idx = defaultdict(lambda : len(cat2idx))
    y_train_idx = [cat2idx[x] for x in y_train]
    y_test_idx = [cat2idx[x] for x in y_test]

    # train
    result_dict = defaultdict(lambda: [])
    best_loss = 1e10
    
    # training
    for sg in [1,0]:
        for sample in [1e-05, 1e-06]:
            for min_count in [1, 5]:
                for alpha in [0.025, 0.1, 0.25]:
                    for vector_size in [100,300]:
                        for negative in [5,15]:
                            for n_epochs in [3,10,30]:
                                print('==============================================================================')
                                print('Model Training Started')
                                
                                # word2vec 모델 생성
                                word2vec = Word2Vec(
                                    sentences = token_ls,
                                    sg = sg,
                                    size = vector_size,
                                    alpha = alpha,
                                    min_count = min_count,
                                    sample = sample,
                                    workers = 4,
                                    negative = negative,
                                    iter = n_epochs,
                                )

                                # CBOW 생성
                                X_train = make_cbow(x_train, word2vec)
                                X_test = make_cbow(x_test, word2vec)
                                
                                # 분류기 학습
                                clf = LogisticRegression(solver = 'sag', multi_class = 'multinomial')

                                clf.fit(X_train, y_train_idx)
                                y_prob = clf.predict_log_proba(X_test)
                                y_pred = [prob.argmax() for prob in y_prob]

                                acc = accuracy_score(y_test_idx, y_pred)
                                loss = log_loss(y_test_idx, y_prob, normalize=False)

                                print('Accuracy : ', acc)

                                # 모델 저장할 폴더 생성
                                if not os.path.exists('Word2Vec_model'):
                                    os.mkdir('Word2Vec_model')

                                # 분류 결과가 best인 모델만 저장
                                if loss < best_loss:
                                    best_loss = loss
                                    word2vec.save('Word2Vec_model/best_w2v_model')
                                
                                # 결과 저장
                                result_dict['sg'].append(sg)
                                result_dict['corpus_count'].append(len(token_ls))
                                result_dict['min_count'].append(min_count)
                                result_dict['vector_size'].append(vector_size)
                                result_dict['n_epochs'].append(n_epochs)
                                result_dict['sample'].append(sample)
                                result_dict['accuracy'].append(acc)

                                # 분류 결과 저장
                                pd.DataFrame(result_dict).to_csv('Word2Vec_tuning_result.csv', index=False)

                                print('Model Training Finished')
    return

if __name__=='__main__':
    main()

# Doc2Vec

In [248]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [249]:
class D2V(object):
    #####################################################
    ##################   Doc2Vec   ######################
    #####################################################

    def __init__(self, path_to_model = ''):
        '''
        기존에 학습된 모델을 불러옵니다.

        Inputs
        =========================
        path_to_model : str,
            학습된 모델이 저장된 path
        '''
        try:
            self.Doc2Vec_model = Doc2Vec.load(path_to_model)
            print('학습된 모델을 성공적으로 불러왔습니다.')
            return
        except:
            print('모델을 불러오지 못하였습니다.')
            print('새로운 모델을 생성해주시기 바랍니다.')
            return


    def make_Doc2Vec_model(self,
                           dm = 1,
                           dbow_words = 0,
                           window = 15,
                           vector_size = 100,
                           sample = 1e-5,
                           min_count = 5,
                           hs = 0,
                           negative = 5,
                           dm_mean = 0,
                           dm_concat = 0):
        '''
        Doc2Vec 모델의 초기설정을 입력하는 함수입니다.
        기존에 만들어진 모델을 load하여 사용할 수 있습니다. (load_Doc2Vec_model 함수를 사용)

        Inputs
         - dm : PV-DBOW / default 1
         - dbow_words : w2v simultaneous with DBOW d2v / default 0
         - window : distance between the predicted word and context words
         - vector size : vector_size
         - min_count : ignore with freq lower
         - workers : multi cpu
         - hs : hierarchical softmax / default 0
         - negative : negative sampling / default 5

        Return
         - None
        '''
        cores = multiprocessing.cpu_count()
        logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

        self.Doc2Vec_model = Doc2Vec(
            dm= dm,                     # PV-DBOW / default 1
            dbow_words= dbow_words,     # w2v simultaneous with DBOW d2v / default 0
            window= window,             # distance between the predicted word and context words
            vector_size= vector_size,   # vector size
            sample = sample,
            min_count = min_count,      # ignore with freq lower
            workers= cores,             # multi cpu
            hs = hs,                    # hierarchical softmax / default 0
            negative = negative,        # negative sampling / default 5
            dm_mean = dm_mean,
            dm_concat = dm_concat,
        )

        return



    def build_and_train_Doc2Vec_model(self,
                                      train_doc_ls,
                                      train_tag_ls,
                                      n_epochs = 10,
                                      if_tokenized = True,
                                      if_morphs = True):

        '''
        Doc2Vec 모델을 생성 혹은 Load한 다음 작업으로, Doc2Vec을 build하고 학습을 수행합니다.

        Inputs
        =================================
        train_doc_ls : iterable,
            documents(tokenized or not tokenized)

        train_tag_ls : iterable,
            tags of each documents

        n_epochs : int
            numbers of iteration

        '''


        # tokenized된 input데이터를 받으면 tokenizing skip
        if if_tokenized :
            train_token_ls = train_doc_ls
        else:
            clean_train_doc_ls = self.clean_doc(train_doc_ls)
            if if_morphs:
                train_token_ls = self.extract_tokens_for_all_document_FAST_VERSION(clean_train_doc_ls, if_morphs = True)
            else:
                train_token_ls = self.extract_tokens_for_all_document_FAST_VERSION(clean_train_doc_ls, if_morphs = False)

        # train_tag_ls를 리스트 형태로 변환 (Series 형태로 들어올 경우)
        try:  train_tag_ls = train_tag_ls.tolist()
        except:   pass

        # words와 tags로 구성된 namedtuple 형태로 데이터 변환 (tagging 작업)
        tagged_train_doc_ls = [TaggedDocument(tuple_[0], [tuple_[1]]) for i, tuple_ in enumerate(zip(train_token_ls, train_tag_ls))]

        # Doc2Vec 모델에 단어 build작업 수행
        self.Doc2Vec_model.build_vocab(tagged_train_doc_ls)

        # 학습 수행
        self.Doc2Vec_model.train(tagged_train_doc_ls,
                                 total_examples= self.Doc2Vec_model.corpus_count,
                                 epochs= n_epochs)
        return


    def train_Doc2Vec_model(self,
                            train_doc_ls,
                            train_tag_ls,
                            n_epochs = 10,
                            if_tokenized = True,
                            if_morphs = True,
                            ):
        '''
        built된 Doc2Vec 모델에 추가적인 학습을 수행합니다.

        Inputs
         - train_doc_ls : iterable, 
            documents(tokenized or not tokenized)

         - train_tag_ls : iterable, 
            tags of each documents

         - n_epochs : int 
            numbers of iteration

         - if_morphs :  boolean
            원문에 대한 tokenizing을 수행할 때, morphs를 추출 (defaulf = True),

         - if_tokenized : boolean 
            True if input document is tokenized [default = True]

         - if_morphs : boolean,
            True : if not tokenized, tokenized with morphs,
            False : if not tokenized, tokenized with nouns.

        Return
         - None
        '''


        # tokenized된 input데이터를 받으면 tokenizing skip
        if if_tokenized :
            train_token_ls = train_doc_ls
        else:
            clean_train_doc_ls = self.clean_doc(train_doc_ls)
            if if_morphs:
                train_token_ls = self.extract_tokens_for_all_document_FAST_VERSION(clean_train_doc_ls, if_morphs = True)
            else :
                train_token_ls = self.extract_tokens_for_all_document_FAST_VERSION(clean_train_doc_ls, if_morphs = False)

        # train_tag_ls를 리스트 형태로 변환 (Series 형태로 들어올 경우)
        try:  train_tag_ls = train_tag_ls.tolist()
        except:   pass

        # words와 tags로 구성된 namedtuple 형태로 데이터 변환 (tagging 작업)
        tagged_train_doc_ls = [TaggedDocument(tuple_[0], [tuple_[1]]) for i, tuple_ in enumerate(zip(train_token_ls, train_tag_ls))]

        # 학습 수행
        self.Doc2Vec_model.train(tagged_train_doc_ls,
                                 total_examples= self.Doc2Vec_model.corpus_count,
                                 epochs= n_epochs)
        return


    # 문서 벡터를 추정하기 위한 함수 (병렬처리에 사용)
    def _infer_vector(self,
            doc_ls,
            alpha=0.1,
            steps=30,
            queue = False):

        '''
        Doc2Vec을 사용하여, documents를 vectorize하는 함수입니다.
        본 함수는 병렬처리를 위해 사용합니다.

        Inputs
        doc_ls : iterable or str
            array of tokenized documents

        alpha : int
        steps : int

        return
         - matrix of documents inferred by Doc2Vec
        '''
        return_ls = []

        # 문서 1개가 들어온 경우,
        if type(doc_ls) == str:
            return self.Doc2Vec_model.infer_vector(doc_ls,
                                                   alpha = alpha,
                                                   min_alpha = self.Doc2Vec_model.min_alpha,
                                                   steps = steps)

        # 복수 개의 문서가 input으로 들어온 경우,
        else:
            return [self.Doc2Vec_model.infer_vector(doc,
                                                    alpha = alpha,
                                                    min_alpha = self.Doc2Vec_model.min_alpha,
                                                    steps = steps) \
                    for doc in doc_ls]


    ###################################################
    ################### 병렬처리 적용 ####################
    ###################################################
    def _multiprocessing_queue_put(self, func, queue, **kwargs):
        queue.put(func(**kwargs))
        return


    def infer_vectors_multiprocessing(self, doc_ls):
        queue_ls = []
        procs = []
        result_ls = []
        batch_size = len(doc_ls) // 10

        # process에 작업들을 할당
        for i, idx in enumerate(range(0, len(doc_ls), batch_size)):
            try:
                batch_ls = doc_ls[idx : idx + batch_size]
            except:
                batch_ls = doc_ls[idx :]

            queue_ls.append(Queue())
            proc = Process(
                    target= self._multiprocessing_queue_put,
                    kwargs = {
                        'func' : self._infer_vector,
                        'queue' : queue_ls[i],
                        'doc_ls' : batch_ls})

            procs.append(proc)
            proc.start()

        for queue in queue_ls:
            temp_result_ls = queue.get()
            queue.close()
            del queue

            result_ls += temp_result_ls

        for proc in procs:
            proc.join()
            del proc

        return np.array(result_ls)

In [250]:
from sklearn.model_selection import train_test_split

In [254]:
token_arr = np.array(token_ls)
category_arr = np.array(category_ls)

In [258]:
# train test split
train_size = int(round(len(token_ls) * 0.8))

x_train, x_test = token_ls[:train_size], token_ls[train_size:]
y_train, y_test = category_ls[:train_size], category_ls[train_size:]


In [259]:
# Y 더미화
cat2idx = defaultdict(lambda : len(cat2idx))
y_train_idx = [cat2idx[x] for x in y_train]
y_test_idx = [cat2idx[x] for x in y_test]

# 모델 저장할 폴더 생성
if not os.path.exists('Doc2Vec_model'):
    os.mkdir('Doc2Vec_model')

# 기존에 학습하던 결과가 있으면 불러온다.
if os.path.isfile('Doc2Vec_model/Doc2Vec_tuning_result.csv'):
    result_dict = pd.read_csv('Doc2Vec_model/Doc2Vec_tuning_result.csv').to_dict('list')
    print('기존의 튜닝 결과를 불러옵니다.')
else:
    result_dict = defaultdict(lambda : [])


In [270]:
from sklearn.manifold import TSNE

In [ ]:
result_dict = defaultdict(lambda : [])
# training
for dm in [1]:
    for sample in [1e-05, 1e-06]:
        for min_count in [1, 5, 15]:
            for vector_size in [100,300]:
                for negative in [5,10,15]:
                    for n_epochs in [10,30]:
                        print('==============================================================================')
                        print('Model Training Started')

                        # Doc2Vec 모델 생성
                        d2v.make_Doc2Vec_model(
                            dm = dm,
                            min_count = min_count,
                            sample = sample,
                            vector_size = vector_size)

                        # build and train Doc2Vec
                        d2v.build_and_train_Doc2Vec_model(x_train, category_ls, n_epochs = n_epochs)
                        
                        '''
                        # 단어 벡터 추정량 분포 시각화
                        X =d2v.infer_vectors_multiprocessing(x_test)

                        tsne= TSNE(n_components=2)
                        X_tsne = tsne.fit_transform(X)
                        scatter_df = pd.DataFrame(X_tsne,
                                                  index = y_test,
                                                  columns = ['x','y'])

                        plt.figure(figsize = (10, 10))

                        for i,section in enumerate(set(category_ls)):
                            temp_df = scatter_df[scatter_df.index == section]
                            plt.scatter(temp_df['x'].values, temp_df['y'].values, label = section, c = np.random.rand(3,))

                        plt.legend(loc = 'best')
                        plt.savefig('추정된 벡터 분포 t-sne ver')

                        '''
                        
                        # 벡터 추정 후 학습 및 성과 평가
                        X_train = d2v.infer_vectors_multiprocessing(x_train)
                        X_test = d2v.infer_vectors_multiprocessing(x_test)

                        # 분류기 학습
                        clf = LogisticRegression(solver = 'sag', multi_class = 'multinomial')

                        clf.fit(X_train, y_train_idx)
                        y_prob = clf.predict_log_proba(X_test)
                        y_pred = [prob.argmax() for prob in y_prob]

                        acc = accuracy_score(y_test_idx, y_pred)
                        loss = log_loss(y_test_idx, y_prob, normalize=False)

                        print('Accuracy : ', acc)

                        # 분류 결과가 best인 모델만 저장
                        if loss < best_loss:
                            best_loss = loss
                            d2v.Doc2Vec_model.save('Doc2Vec_model/best_d2v_model')
                        
                        # 결과 저장
                        result_dict['dm'].append(dm)
                        result_dict['corpus_count'].append(len(token_ls))
                        result_dict['min_count'].append(min_count)
                        result_dict['vector_size'].append(vector_size)
                        result_dict['negative'].append(negative)
                        result_dict['n_epochs'].append(n_epochs)
                        result_dict['sample'].append(sample)
                        result_dict['accuracy'].append(acc)
                        result_dict['loss'].append(loss)

                        # 분류 결과 저장
                        pd.DataFrame(result_dict).to_csv('Doc2Vec_model/Doc2Vec_tuning_result.csv', index=False)

                        print('Model Training Finished')